## Our Script so far

In [1]:
from csv import reader
import dataset
import pickle

data_rdr = reader(open('mn.csv', 'r', encoding='utf-8'))
header_rdr = reader(open('mn_headers_updated.csv', 'r', encoding='utf-8'))

data_rows = [d for d in data_rdr]
header_rows = [h for h in header_rdr if h[0] in data_rows[0]]

all_short_headers = [h[0] for h in header_rows]
skip_index = []
final_header_rows = []

for header in data_rows[0]:
    if header not in all_short_headers:
        print(header)
        index = data_rows[0].index(header)
        if index not in skip_index:
            skip_index.append(index)
    else:
        for head in header_rows:
            if head[0] == header:
                final_header_rows.append(head)
                break
new_data = []

for row in data_rows[1:]:
    new_row = []
    
    for i, d in enumerate(row):
        if i not in skip_index:
            new_row.append(d)
    new_data.append(new_row)

zipped_data = []
for drow in new_data:
    zipped_data.append(zip(final_header_rows, drow))
    
# Save the list to a file
with open('zipped_data.pkl', 'wb') as file:
    pickle.dump(zipped_data, file)


mwelevel
mnweight
wscoreu
windex5u
wscorer
windex5r


In [2]:
# Load the list from the file
def load_zipped_data():
    with open('zipped_data.pkl', 'rb') as file:
        zipped_data = pickle.load(file)
        return zipped_data

In [3]:
# look for missing
zipped_data = load_zipped_data()
for x in zipped_data[0]:
    if not x[1]:
        print(x)

In [4]:
#look for duplicates

set_of_keys = set()
zipped_data = load_zipped_data()
for x in zipped_data:
    unique_key = ''
    for i, row in enumerate(x):
        if i in [0, 1, 2]:
            unique_key += str(row[1])+'-'
    set_of_keys.add(unique_key[:-1])

uniques = []
zipped_data = load_zipped_data()
for x in zipped_data:
    unique_key = ''
    for i, row in enumerate(x):
        if i in [0, 1, 2]:
            unique_key += str(row[1])+'-'
    if not set_of_keys.remove(unique_key[:-1]):
        uniques.append(x)
        
print(set_of_keys)

set()


In [5]:
#save to db
zipped_data = load_zipped_data()
db = dataset.connect('sqlite:///data_wrangling.db')
table = db['unicef_survey']

for row_num, data in enumerate(zipped_data):
    for question, answer in data:
        data_dict = {
            'question': question[1],
            'question_code': question[0],
            'answer': answer,
            'response_number': row_num,
            'survery': 'mn',
        }
    table.insert(data_dict)

### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Refactoring the previous Script

### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### We can see most of our code is flat, meaning we don't have nested levels of importance. Much of the code and functions sit without identation or documentation in the file. It's not well abstracted, and the variable names are unclear. let's start working on parts of that, beginning at the top. The first two sets of lines repeat each other. Let's write a function to do that instead.

In [1]:
from csv import reader
import pickle
import dataset

In [2]:
def get_rows(file_name):
    rdr = reader(open(file_name, 'r', encoding='utf-8'))
    return [row for row in rdr]

In [3]:
def eliminate_mismatches(header_rows, data_rows):
    all_short_headers = [h[0] for h in header_rows]
    skip_index = []
    final_header_rows = []

    for header in data_rows[0]:
        if header not in all_short_headers:
            print(header)
            index = data_rows[0].index(header)
            if index not in skip_index:
                skip_index.append(index)
        else:
            for head in header_rows:
                if head[0] == header:
                    final_header_rows.append(head)
                    break
    return skip_index, final_header_rows

In [4]:
def zip_data(headers, data):
    zipped_data = []
    for drow in data:
        zipped_data.append(zip(headers, drow))
    return zipped_data

In [5]:
def create_zipped_data(final_header_rows, data_rows, skip_index):
    new_data = []

    for row in data_rows[1:]:
        new_row = []

        for index, data in enumerate(row):
            if index not in skip_index:
                new_row.append(data)
        new_data.append(new_row)
    zipped_data = zip_data(final_header_rows, new_data)
    # Save the list to a file
    with open('zipped_data.pkl', 'wb') as file:
        pickle.dump(zipped_data, file)
        
    return zipped_data

In [6]:
# Load the list from the file
def load_zipped_data():
    with open('zipped_data.pkl', 'rb') as file:
        zipped_data = pickle.load(file)
        return zipped_data

### with our new functions, we were able to preserve our code, clear up some variable names, and add a helper function to zip headers with rows of data and return the list of zipped data. The code is clearer and broken up more appropriately. We're going to continue to apply the same logic to the rest of the file:

In [7]:
def find_missing_data(zipped_data):
    missing_count = 0
    for row in zipped_data:
        for question, answer in row:
            if not answer:
                missing_count += 1
    return missing_count

In [10]:
def find_duplicate_data(zipped_data):
    set_of_keys = set()
    for x in zipped_data:
        unique_key = ''
        for i, row in enumerate(x):
            if i in [0, 1, 2]:
                unique_key += str(row[1])+'-'
        set_of_keys.add(unique_key[:-1])

    uniques = []
    zipped_data = load_zipped_data()
    for x in zipped_data:
        unique_key = ''
        for i, row in enumerate(x):
            if i in [0, 1, 2]:
                unique_key += str(row[1])+'-'
        if not set_of_keys.remove(unique_key[:-1]):
            uniques.append(x)

    return uniques, len(set_of_keys)

In [11]:
def save_to_sqlitedb(db_file, zipped_data, survey_type):
    db = dataset.connect(db_file)
    table = db['unicef_survey']
    all_rows = []

    for row_num, data in enumerate(zipped_data):
        for question, answer in data:
            data_dict = {
                'question': question[1],
                'question_code': question[0],
                'answer': answer,
                'response_number': row_num,
                'survery': 'mn',
            }
            all_rows.append(data_dict)
            
    table.insert_many(all_rows)

### We need to now work on re-creating how to use all these setps in a main function:

In [12]:
def main():
    data_rows = get_rows('mn.csv')
    header_rows = get_rows('mn_headers_updated.csv')
    skip_index, final_header_rows = eliminate_mismatches(header_rows, data_rows)
    zipped_data = create_zipped_data(final_header_rows, data_rows, skip_index)
    num_missing = find_missing_data(zipped_data)
    uniques, num_dupes = find_duplicate_data(load_zipped_data())
    if num_missing == 0 and num_dupes == 0:
        save_to_sqlitedb('sqlite:///database/data_wrangling.db', load_zipped_data(), 'mn')
    else:
        error_msg = ''
        if num_missing:
            error_msg += 'We are missing {} values. '.format(num_missing)
        if num_dupes:
            error_msg += 'We have {} duplicates. '.format(num_dupes)
        error_msg += 'Please have a look and fix!'
        print(error_msg)

In [13]:
main()


mwelevel
mnweight
wscoreu
windex5u
wscorer
windex5r
